In [152]:
import pandas as pd
# define options for suits
suits = ['H','D','S','C']
# define options for values
# Aces are notated as 1s, Jacks are notated as 11s, Queens are notated as 12s, Kings are notated as 13s
values = list(range(1,14))
# use list comprehensions to get a data frame of all possible cards
# each suit repeated 13 times; list of values repeated 4 times
cards_df = pd.DataFrame(data = {'suit':[y for z in [13*[x] for x in suits] for y in z],'value':values*4})
print(cards_df.head(3)) # print first 3 rows
print(cards_df.tail(3)) # print last 3 rows

  suit  value
0    H      1
1    H      2
2    H      3
   suit  value
49    C     11
50    C     12
51    C     13


In [153]:
# create empty list to store all possible cards
cards_list = []
# for each row in the data frame, define the card as a list of [suit,value]
# then append the card to the list
for i in range(0,len(cards_df)):
    card = [cards_df.iloc[i].suit,cards_df.iloc[i].value]
    cards_list.append(card)
print(cards_list) # show the resulting list

[['H', 1], ['H', 2], ['H', 3], ['H', 4], ['H', 5], ['H', 6], ['H', 7], ['H', 8], ['H', 9], ['H', 10], ['H', 11], ['H', 12], ['H', 13], ['D', 1], ['D', 2], ['D', 3], ['D', 4], ['D', 5], ['D', 6], ['D', 7], ['D', 8], ['D', 9], ['D', 10], ['D', 11], ['D', 12], ['D', 13], ['S', 1], ['S', 2], ['S', 3], ['S', 4], ['S', 5], ['S', 6], ['S', 7], ['S', 8], ['S', 9], ['S', 10], ['S', 11], ['S', 12], ['S', 13], ['C', 1], ['C', 2], ['C', 3], ['C', 4], ['C', 5], ['C', 6], ['C', 7], ['C', 8], ['C', 9], ['C', 10], ['C', 11], ['C', 12], ['C', 13]]


In [154]:
import itertools
# make a list of all possible 5 card combinations
hands_list = list(itertools.combinations(cards_list,5))
print(hands_list[:3]) # display first 3
print(hands_list[len(hands_list)-3:]) # display last 3

[(['H', 1], ['H', 2], ['H', 3], ['H', 4], ['H', 5]), (['H', 1], ['H', 2], ['H', 3], ['H', 4], ['H', 6]), (['H', 1], ['H', 2], ['H', 3], ['H', 4], ['H', 7])]
[(['C', 8], ['C', 9], ['C', 11], ['C', 12], ['C', 13]), (['C', 8], ['C', 10], ['C', 11], ['C', 12], ['C', 13]), (['C', 9], ['C', 10], ['C', 11], ['C', 12], ['C', 13])]


In [155]:
# create a user-defined function (UDF) to convert a 5 card hand into the "category" of hand
def hand_category(hand):
    # get a list of the suits in the hand
    hand_suits = [x[0] for x in hand]
    # create a list of number of each suit in the hand [hearts,diamonds,spades,clubs]
    suit_counts = [hand_suits.count(x) for x in suits]
    # get a list of the values in the hand
    hand_values = [x[1] for x in hand]
    # sort the values numerically, ascending
    hand_values.sort()
    # create a list of the number of each value in the hand [1s,2s,3s,...,11s,12s,13s]
    value_counts = [hand_values.count(x) for x in values]
    # create a list of each increase in value [value1-value0,value2-value1,value3-value2,value4-value3]
    # note that this makes a four-item list from the five-item list of values
    value_inc = [hand_values[i]-hand_values[i-1] for i in range(1,5)]
    # define the conditions for each type of hand, working from the best hand type down
    if (sum([x==1 for x in value_inc])==4 or hand_values == [10,11,12,13,1]) and max(suit_counts) == 5:
        category = "straight-flush"
    elif max(value_counts) == 4:
        category = "four-of-a-kind"
    elif sum([x==3 for x in value_counts]) == 1 and sum([x==2 for x in value_counts]) == 1:
        category = "full-house"
    elif max(suit_counts) == 5:
        category = "flush"
    elif sum([x==1 for x in value_inc])==4 or hand_values == [10,11,12,13,1]:
        category = "straight"
    elif max(value_counts) == 3:
        category = "three-of-a-kind"
    elif sum([x==2 for x in value_counts]) == 2:
        category = "two-pair"
    elif max(value_counts) == 2:
        category = "pair"
    else:
        category = "high-card"
    return category
    # the udf will return the category evaluated through the nested if-elif-else loop

In [157]:
# time the codes
from time import time
t0=time()

# create a data frame of each hand and the corresponding category
# use list comprehension to run the hand_category udf on each hand
hands_df = pd.DataFrame(data={'hand':hands_list,'category':[hand_category(h) for h in hands_list]})

# print the elapsed time
print("time to run:",round(time()-t0,2),"seconds")
# print the first 5 rows of the data frame
hands_df.head()

time to run: 38.97 seconds


,hand,category
0,"([H, 1], [H, 2], [H, 3], [H, 4], [H, 5])",straight-flush
1,"([H, 1], [H, 2], [H, 3], [H, 4], [H, 6])",flush
2,"([H, 1], [H, 2], [H, 3], [H, 4], [H, 7])",flush
3,"([H, 1], [H, 2], [H, 3], [H, 4], [H, 8])",flush
4,"([H, 1], [H, 2], [H, 3], [H, 4], [H, 9])",flush


In [159]:
# get the total number of possible hands
total_hands = hands_df.count().iloc[0]
print("total possible hands:",total_hands)
# get the total number of hands in each category
category_counts = hands_df.category.value_counts().sort_values()
# get the probabilities
probs = category_counts/total_hands
print(probs)
# double check that probabilities add up to 1
print("total:",sum(category_counts/total_hands))

total possible hands: 2598960
category
straight-flush     0.000014
four-of-a-kind     0.000240
full-house         0.001441
flush              0.001967
straight           0.003532
three-of-a-kind    0.021128
two-pair           0.047539
pair               0.422569
high-card          0.501570
Name: count, dtype: float64
total: 1.0
